## Dataset generation using opencv

In [1]:
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
import keras
import keras_metrics
import matplotlib.pyplot as plt
import os 
from random import shuffle
from tqdm import tqdm

C:\Users\Marhaba\Anaconda3\envs\New_Environment_For_Keras\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Marhaba\Anaconda3\envs\New_Environment_For_Keras\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Marhaba\Anaconda3\envs\New_Environment_For_Keras\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\User

In [ ]:
def dataset_generation():
    FaceClassifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_detection(image):
        gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        detected_face = FaceClassifier.detectMultiScale(gray_scale, 1.3, 5)
        
        if detected_face is ():
            return None
        for (x,y,w,h) in detected_face:
            face = image[y:y+h,x:x+w]
        return face
    
    videoCapture = cv2.VideoCapture(0)
    image_id = 0
    
    while True:
        ret, frame = videoCapture.read()
        if face_detection(frame) is not None:
            image_id+=1
            face = cv2.resize(face_detection(frame), (250,250))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            #Saving images from web cam to local disk
            file_name_path = "Dataset/"+"Yousaf_"+str(image_id)+'.jpg'
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(image_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2 )
            
            cv2.imshow("Cropped_Face", face)
            if cv2.waitKey(1)==13 or int(image_id)==500:
                break
                
    videoCapture.release()
    cv2.destroyAllWindows()
    print("Data Collection is completed")

In [ ]:
dataset_generation()

In [2]:
import numpy as np

In [3]:
#Creating labels for each person
def label(image):
    name = image.split("_")[0]
    #One hot encoding for four persons 
    if name == "Yousaf":
        return 0
    elif name == "Qazi":
        return 1
    elif name == "Abdullah":
        return 2
    elif name == "Manzoor":
        return 3

## Preparing dataset for deep learning model

In [4]:
def load_data():
    dataset = []
    for image_name in tqdm(os.listdir("C:\\Users\\Marhaba\\Desktop\\Computer Vision\\FaceId_Door_Lock_System\\Dataset")):
        image_path = os.path.join("C:\\Users\\Marhaba\\Desktop\\Computer Vision\\FaceId_Door_Lock_System\\Dataset",image_name)
        image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        dataset.append([image,label(image_name)])
    shuffle(dataset)
    return dataset

In [5]:
dataset = load_data()

100%|██████████| 2000/2000 [00:01<00:00, 1222.39it/s]


In [6]:
dataset[1]

[array([[130, 130, 130, ...,  47,  48,  48],
        [131, 131, 130, ...,  48,  48,  49],
        [132, 132, 131, ...,  48,  48,  48],
        ...,
        [ 59,  58,  57, ...,  24,  23,  23],
        [ 59,  58,  57, ...,  24,  23,  23],
        [ 59,  58,  57, ...,  23,  23,  23]], dtype=uint8),
 3]

In [7]:
#Splitting dataset into training and testing 
train_dataset = dataset[:1600]
test_dataset = dataset[1600::]
X = np.array([k[0] for k in train_dataset]).reshape(-1,250,250,1)
print(X.shape)
Y = [j[1] for j in train_dataset]
Y = to_categorical(Y)
X_test = np.array([k[0] for k in test_dataset]).reshape(-1,250,250,1)
print(X_test.shape)
Y_test = [j[1] for j in test_dataset]
Y_test = to_categorical(Y_test)


(1600, 250, 250, 1)
(400, 250, 250, 1)


## CNN architecture for face recognition

In [ ]:
print(Y[0])

In [13]:
#CNN Architecture for face recognition
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same", input_shape=(250,250,1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(1048, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 250, 250, 64)      1664      
_________________________________________________________________
batch_normalization_17 (Batc (None, 250, 250, 64)      256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 124, 124, 64)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 124, 124, 128)     73856     
_________________________________________________________________
batch_normalization_18 (Batc (None, 124, 124, 128)     512       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 124, 124, 64)      73792     
_________________________________________________________________
batch_normalization_19 (Batc (None, 124, 124, 64)     

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',keras_metrics.precision(),keras_metrics.recall(),keras_metrics.f1_score()])
history=model.fit(X, Y, epochs=10, batch_size=32, verbose=1, validation_split=0.3)

tracking <tf.Variable 'Variable:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_1:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_2:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_3:0' shape=() dtype=int32> fn
tracking <tf.Variable 'Variable_4:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_5:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_6:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_7:0' shape=() dtype=int32> fn

Train on 1120 samples, validate on 480 samples
Epoch 1/10


In [ ]:
model.evaluate(X_test,Y_test,verbose=1)

In [ ]:
history.history.keys()


In [ ]:
print(history.history.keys())
plt.plot(history.history['precision'])
plt.plot(history.history['recall'])
plt.plot(history.history['f1_score'])
plt.title('Precision Recall')
plt.ylabel('Precision')
plt.xlabel('Number of epocs')
plt.legend(['Precision', 'Recall','F1 Score'], loc='upper left')
plt.show()

In [ ]:
model.save('Saved_Model.h5')


In [ ]:
# saving and loading the model weights
 
# save model
model.save_weights('gfgModelWeights')
print('Model Saved!')
 
# load model
savedModel = model.load_weights('gfgModelWeights')
print('Model Loaded!')

In [9]:
#CNN Architecture for face recognition
model2 = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same", input_shape=(250,250,1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Flatten(),
    keras.layers.Dense(1048, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(4, activation='softmax')
])
model2.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 60, 60, 32)        3904      
_________________________________________________________________
batch_normalization_4 (Batch (None, 60, 60, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 29, 29, 64)        51264     
_________________________________________________________________
batch_normalization_5 (Batch (None, 29, 29, 64)        256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)     

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',keras_metrics.precision(),keras_metrics.recall(),keras_metrics.f1_score()])
# load model
savedModel = model2.load_weights('gfgModelWeights')
print('Model Loaded!')

In [ ]:
model2.evaluate(X_test,Y_test,verbose=1)

In [ ]:
def prediction_model():
    FaceClassifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_detection(image):
        gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        detected_face = FaceClassifier.detectMultiScale(gray_scale, 1.3, 5)
        
        if detected_face is ():
            return None
        for (x,y,w,h) in detected_face:
            face = image[y:y+h,x:x+w]
        return face
    
    videoCapture = cv2.VideoCapture(0)    
    while True:
        ret, frame = videoCapture.read()
        if face_detection(frame) is not None:
            face = cv2.resize(face_detection(frame), (250,250))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            cv2.putText(face, str("User"), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2 )
            cv2.imshow("Cropped_Face", face)
            prediction = model2.predict_classes(face.reshape(-1, 250, 250, 1))
            print(prediction)
            if cv2.waitKey(1)==13:
                break
                
    videoCapture.release()
    cv2.destroyAllWindows()
    print("Data Collection is completed")

In [ ]:
prediction = model2.predict_pr(X[19].reshape(-1, 250, 250, 1))

In [ ]:
print(prediction)

In [ ]:
prediction_model()